Housekeeping:  
This is code using python code for ArcGIS using the specific arcpy packages    
  
If running in the notebook function within the ArcGIS software, you dont need to do anything special.   
  
If you would like to run in Visual Studio Code:  
Press Control + Shift + P and search for “>Python: Select Interpreter”.  
The command palette will drop down, and you should see some options to seelct. Choose the first option when it says, “Enter interpreter path…”:  
TO find your interpreter path in its default location go to the following path or select the following path in the dropdown menu:  
    C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\  
  
further instructions here: https://resources.esri.ca/getting-technical/how-to-configure-visual-studio-code-with-arcgis-pro-s-python-environment  



# Code Sections  
*There are sections that you will need to go and edit or configure based on your computer and spatial requirements - see chunk below this*    
[Set Workspaces](#workspaces)  
[Layers Required](#layersrequired)  
[Layers Created](#layerscreated)  
  
[1. Setting Up The Data](#editing)   
   > [Setting File Names](#filenames)   
   > [Setting Specific Paramters](#parameters)  
     > Forest Cover  
     > Road Buffer Distance  
     > Elevation Cutoff values  
     > [Projected Coordinate System](#coordinates)  
     > Geographic Coordinate System  
           
[2. Manipulating Downloaded Data](#manipulations) 
   > [Moving data Layers to gdb](#movelayers)  
   > [Compiling ESA land cover tiles](#esacompile)   
  
  
[3. Generate Forest Cover from ESA WorldCover](#ForestCover)    
  
[4. Generate Access layer](#AccessLayer)    
   > [4a. Distance from Coast](#distcoast)  
   > [4b. Roads](#roads)  
   > [4c. Slope](#slope)  
   > [4d. Elevation](#elevation)  
   > [4e. Adjacent to Modified Land](#modifiedland)  
   > [4f. Combine Access Layers](#combineaccess)  
   
[5. Combing Forest Cover * Access](#CombineFC_Access)  

 



# Code Sections you *MUST* EDIT   
or at least check to make sure they are correct  
[Set New Geodatabase Folder](#new_gdb)  (only do on the first time running this code)  
[Set Folder Location for Raw Data](#raw_data)  
[Set Workspaces](#workspaces)  
[Setting Layer Names](#filenames)   
[Setting Parameter Values](#parameters)  
[Setting Coordinate Systems](#coordinates)  

# Forest Exploitation Potential (ExPot) Workflow

These layers measure the intensity that a resource (in this case, forest) is used or has the potential to be used, modified, or extracted.  
high exploitation potential values = high intensity use or potential of use
low exploitation potential values  = low intensity use (or low likelihood or resource use)  


1. Generate Resource layer
          > ESA Land Cover Layer - turn forest and mangrove into "Forest"
2. Generate Access layer, which combines:  
          > 2a. Distance from Coast  
          > 2b. Roads  
          > 2c. Slope  
          > 2d. Elevation  
          > 2e. Areas Adjacent to Modified Land    
          > Combine Access Layers    
3. Ensure 'Forest Area' and 'Access Layers' are scaled (0-100)  
4. Combine 'Forest Area' x 'Access Layers' by multiplying   
          > by multiplying, this ensures that ONLY ExPot values are present in areas where there is forest resource  
          > you cannot have the potential of forest extraction in areas where there is not forest (i.e. deforested land, urban area etc)
5. Rescale final 'forest impact' hazard layer to 0-100


<a id= "workspaces"></a>

#  Set Workspaces
Set variable names  
Put all your downloaded data into one file and load into GIS.  
Put data layers into their own files (e.g. ESA into the ESA_LandCover)  
this will be helpful when you have raster layers that may change by year or location  

In [2]:
from arcgis.gis import arcgis
import arcpy
import os
from arcpy.ia import * #so you can run the Con tool 

## Setting new geodatabase folder to save new layers
<a id= "new_gdb"></a>

In [2]:
#creating a new workspace gdb file to put files into
#-----------------------------------------------------------------------
#If this is the first time running this code, run the createfilegdb line
#otherwise, keep commented out if you are continuing your work 
#-----------------------------------------------------------------------

#Notice here, that the file path is routed to a folder on your computer. It creates a few folder geodatabase called Forest_ExPot
#arcpy.management.CreateFileGDB(r"C:\Users\jc446202\OneDrive - James Cook University (1)\ACIAR_spatial\modified_dat", "Forest_ExPot", "CURRENT")


<Result 'C:\\Users\\jc446202\\OneDrive - James Cook University (1)\\ACIAR_spatial\\modified_dat\\Forest_ExPot.gdb'>

## Setting folder for downloaded raw data files
<a id= "raw_data"></a>

In [3]:
#create file path directory   
# to where you have saved your downloaded data  
#When running through this code, the new files will save into the gdb folder created above

#this is naming the folder pathway "downloaded_data" so you can easily reference it in this code.   
#you need to change the path to where you have saved the downloaded data
downloaded_data = r"C:/Users/jc446202/OneDrive - James Cook University (1)/ACIAR_spatial/modified_dat/forest_impact_files"

## Setting your workspaces
<a id= "workspaces"></a>

In [4]:
#setting your workspaces  
#so you dont have to reference your file paths again for the rest of this code. do it once here         
gdb_workspace = r"C:/Users/jc446202/OneDrive - James Cook University (1)/ACIAR_spatial/modified_dat/Forest_ExPot.gdb"             
# r means read the raw file path name and doesnt interpret \ as python syntax fxns

In [4]:
# now you can use:
#arcpy.env.workspace = gdb_workspace  #to set your current workspace to the gdb folder - for when you want to save new layers 
arcpy.env.workspace = downloaded_data #for when you want to work with the raw files - this will not be used very often, but only to call the raw files if needed

In [5]:
#check workspace
print(arcpy.env.workspace)
print(gdb_workspace)
print(downloaded_data)

C:/Users/jc446202/OneDrive - James Cook University (1)/ACIAR_spatial/modified_dat/forest_impact_files
C:/Users/jc446202/OneDrive - James Cook University (1)/ACIAR_spatial/modified_dat/Forest_ExPot.gdb
C:/Users/jc446202/OneDrive - James Cook University (1)/ACIAR_spatial/modified_dat/forest_impact_files


<a id= "layersrequired"></a>

# Layers required


**these files should be saved in a folder that only keeps downloaded data**  
set the downloaded data file path in the cell above

Shapefiles should be projected to WGS 1984 PDC Mercator  
Load data to Arc Project (drag and drop from file or Add Data)  
All raw downloaded files are in "Forest_Impact_Files"  
You should only have *ONE* folder. All these files get put in that folder and will run.  
if you want to expand to another region, or do this for multiple years, those will need to be in separate folders  
<br>
   
1. Solomon Islands country vector file, projected  
name whatever you like, you will change the name later in the code  
    - "Solomons_country_p"  
<br>  
2. Solomon Islands country LINE file, projected  
name whatever you like, you will change the name later in the code  
if you dont have this, just convert the country polygon to a line
    - "Solomons_line_p"  
<br>  
3. Roads - from open street maps   
https://planet.openstreetmap.org/  
https://solomonislands-data.sprep.org/dataset/openstreetmap-data-solomon-islands
use the osm_roads_free data  
clipped for the Solomon Islands, and projected  
name whatever you like, you will change the name later in the code  
    - "osm_SI_roads_p"  
<br>   
4. DEM data  
https://solomonislands-data.sprep.org/dataset/aster-global-digital-elevation-model-gdem-version-3-astgtm-solomon-islands  
clipped to solomon islands, and projected, 31m resolution  
name whatever you like, you will change the name later in the code  
    - "si_dem"   

5. Land Use Map  
ESA World Cover Map, 10 m resolution, annual updates, 22 thematic classes  
https://esa-worldcover.org/en  
Need to download all tile covering solomon islands  
Keep the file names the defaiult downloaded name:  
    > ESA_WorldCover_10m_2020_v100_S12E165_Map.tif    
    > ESA_WorldCover_10m_2020_v100_S12E162_Map.tif    
    > ESA_WorldCover_10m_2020_v100_S12E159_Map.tif    
    > ESA_WorldCover_10m_2020_v100_S12E156_Map.tif    
    > ESA_WorldCover_10m_2020_v100_S09E162_Map.tif      
    > ESA_WorldCover_10m_2020_v100_S09E159_Map.tif    
    > ESA_WorldCover_10m_2020_v100_S09E156_Map.tif    
    > ESA_WorldCover_10m_2020_v100_S09E153_Map.tif  
     
ESA World Cover Color Legend  
Legend to go with ESA land cover layer.   
  
    - 10 = Tree Cover  
    - 20 = Shrubland  
    - 30 = Grassland  
    - 40 = Cropland  
    - 50 = Built Up  
    - 60 = Bare/Sparse Vegetation  
    - 70 = Snow and Ice  
    - 80 = Permanent Water Bodies  
    - 90 = Herbaceous Wetland  
    - 95 = Mangroves  
  

<a id= "layerscreated"></a>

# Layers Created


## Layers Downloaded:  (in forest impact files)

These are the same layers that get copied over to geodatabase file (except for the ESA_Worldcover one - which are tiled to create the esa2020_si)  

| Layer_Name | File_Type | Description |
|    :---    |  :---     |  :---   |
|**esa2020_si**   | Raster  |  land cover layer from ESA, for Solomons  |
|**ESA_WorldCover_10m_2020_v100_S09E153_Map.tif** | Raster | Land cover layer of ESA tile |
|**ESA_WorldCover_10m_2020_v100_S09E156_Map.tif** | Raster | Land cover layer of ESA tile |
|**ESA_WorldCover_10m_2020_v100_S09E159_Map.tif** | Raster | Land cover layer of ESA tile |
|**ESA_WorldCover_10m_2020_v100_S09E162_Map.tif** | Raster | Land cover layer of ESA tile |
|**ESA_WorldCover_10m_2020_v100_S12E156_Map.tif** | Raster | Land cover layer of ESA tile |
|**ESA_WorldCover_10m_2020_v100_S12E159_Map.tif** | Raster | Land cover layer of ESA tile |
|**ESA_WorldCover_10m_2020_v100_S12E162_Map.tif** | Raster | Land cover layer of ESA tile |
|**ESA_WorldCover_10m_2020_v100_S12E165_Map.tif** | Raster | Land cover layer of ESA tile |
|**osm_SI_roads_p.shp** | shapfile, line | Open Street Map Roads layer |
|**si_dem** | raster | DEM elevation |
|**Solomons_Country_p.shp** | shapfile, polygon | Country polygon of islands |
|**solomons_line.shp** | shapfile, line | Outline of islands |




##   Forest Resource Layers

| Layer_Name | File_Type | Description |
|    :---    |  :---     |  :---   |
|**forest_class**      | Raster  |    classified forest layers, where 1 = forest or mangrove, 0 = everything else  |
|**esa2020_si**   | Raster  |  land cover layer from ESA, for Solomons  |


## Access Measures  

Distance Measures  
  
| Distance Measures | File_Type | Description |
|    :---           |  :---     |  :---       |
|**d_coast**        | Raster    |  distance to coast - going both on land and out to sea out to 25 km  |
|**d_coast_land**   | Raster    |  distance to coast on land only  |
|**d_coast_scale**  | Raster    |  scaled distance to coast on land. close to coast = 100, far from coast = 0  |

Road Measures 
   
| Road Measures | File_Type | Description |
|    :---       |  :---     |  :---       | 
|**osm_SI_roads_p_shp**            | shapefile, line  |  open street map of SI roads, projected  |
|**roads_buff_1k**                 | shapefile  |  500 m either side buffer of roads, total to 1km swath  |
|**roads_buff_1k_rast**            | Raster     |  500 m either side of buffer, 1 km swath raster | 


Slope/Elevation Measures  
  
| Slope/Elevation Measures | File_Type | Description |
|    :---         |  :---     |  :---       |  
|**slope**     | Raster  | slope raster from si_dem  |
|**slope_class**      | Raster  | classified slope values  0-21deg = 100, 21-31 = 50, >31 = 0  |
|**evelation_class**  | Raster  | classified elevation where values >400 m elevation are no access (0), and under 400 m are high access (100)  |
  
Adjancent Habitat Measures  
  
| Adjacent habitat Measures | File_Type | Description |
|    :---         |  :---     |  :---       |  
|**esa2020_si**                        | Raster   |  land cover layer from ESA, for Solomons  |
|**LandCover_poly**                    | polygon  |  polygon layer of esa land cover types  |
|**modified_land**                     | Polygon  |  deforested, cropland, grassland, and built up area polygons  |
|**areas_adjacent_to_modified_land**   | polygon  |  200 m buffer area around all modified lands    |
|**areas_adjacent_to_modified_land**   | raster   |  raster of 200m buffer area around modified lands |
  
Access Measures  
  
| Access Measures | File_Type | Description |
|    :---         |  :---     |  :---       |   
|**forest_access_all**         | Raster  | access to forest resources, the combination of distance to coast, road buffer, slope class, and modified land buffer  |
|**forest_access_scale**   | Raster  | scaled forest access 100 - 0. 100 = high access, 0 = no access.  |


Potential Exploitation
| PotExp Measures | File_Type | Description |
|    :---         |  :---     |  :---       |   
|**forest_potential_exploitation**         | Raster  | forest_class x forest_access_scale. multiplied bc there cannot be potential exploitation where there is not forest  |
|**forest_access_scale**                   | Raster  | scaled forest potential exploitation 100 - 0. 100 = high potexp, 0 = no potexp.  |




## Forest Impact

| Impact Measures | File_Type | Description |
|    :---         |  :---     |  :---       |  
|**forest_impact**         | Raster  | combined forest_scale * forest_access_scale rasters. product.  |
|**forest_impact_scale**   | Raster  | scaled forest access value 0-100.  |

<a id= "editing"></a>


# 1. Parameters Needing Editing

You need to:  
- and/or define certain parameters that might influence the Hazard output.  
- Listed below are default values, but can be changed if desired   
- These values will vary based on expert and informed knowledge about ecological, biophysical and social parameters  

### Need to rename the following layers:
- the polygon of the country outline
- the line file of the country outline
- roads layer
- dem elevation raster layer


### Need to set a value for the following parameters

    
**"road_buff_dist"**     
    Default =  "500 Meters"  
    the distance that is relevant to create access to forested areas adjacent to roads  
    
**"elevation_cutoff"**  
    Default = 400 meters  
    areas above 400 m elevation cannot be logged.  
    
**project_coord**  
    Default = WGS_1984_PDC_Mercator
    the projects coordinate system to use for this analysis
      
**gcs_coord**  
    Default = GCS_WGS_1984
    the geographic coordinate system. in case you need to convert

<a id= "filenames"></a>

## Setting file names

In [6]:
#HERE you need to CHANGE THE NAMES OF THE FILES on the right of the = sign 
# name you want to keep = name you have your data currently saved as
# using the names of the files you have downloaded
# only need to change the below files
# do *NOT* change the file names for the land cover (keep them as default when downloaded)

#changing downloaded shapefile to a consistent name
country_polygon = 'Solomons_country_p'            #this is where you will change the name of the shapefile you have downloaded (The right hand side of the "=" sign) 
country_line = 'solomons_line'
roads = 'osm_SI_roads_p'

#change DEM layer to a consistent name
DEM = 'si_dem'


<a id= "parameters"></a>

## Setting Parameters

These are values you need to change depending on your preferences or relevant literature. 
if unsure, keep them as the default.

In [7]:

#Road buffer distance
road_buff_dist = "500 Meters" #Default is '500 Meters' (1 km swath total)


In [8]:
#elevation cutoff

elevation_cutoff = 400 #default is 400 m


<a id= "coordinates"></a>

## Setting Coordinate Systems

You need to set projected and geographic coordinate systems so all your files are in the same system.  
For solomons keep as defauly below unless you have a specific reason to change these. 

In [36]:
#Projected coordinate system

project_coord = arcpy.SpatialReference('WGS_1984_PDC_Mercator')  #default = 'WGS_1984_PDC_Mercator'
project_coord

name (Projected Coordinate System),WGS_1984_PDC_Mercator
factoryCode (WKID),3832
linearUnitName (Linear Unit),Meter
name (Geographic Coordinate System),GCS_WGS_1984
factoryCode (WKID),4326
angularUnitName (Angular Unit),Degree
datumName (Datum),D_WGS_1984


In [10]:
#Geographic coordinate system

gcs_coord = arcpy.SpatialReference('GCS_WGS_1984') #default = 'GCS_WGS_1984'
gcs_coord

name (Geographic Coordinate System),GCS_WGS_1984
factoryCode (WKID),4326
angularUnitName (Angular Unit),Degree
datumName (Datum),D_WGS_1984


<a id= "manipulations"></a>

# 2. Manipulating downloaded data 

<a id= "esacompile"></a>


## Compiling ESA Land Cover data

In [15]:
#list all ESA rasters in the downloaded data file (forest_impact_files)
arcpy.env.workspace = downloaded_data
ESA_List = arcpy.ListRasters("ESA*") # * is a wild cards. says select anything that has "ESA..." and any value after that.
print(ESA_List)
print(len(ESA_List))

['ESA_WorldCover_10m_2020_v100_S09E153_Map.tif', 'ESA_WorldCover_10m_2020_v100_S09E156_Map.tif', 'ESA_WorldCover_10m_2020_v100_S09E159_Map.tif', 'ESA_WorldCover_10m_2020_v100_S09E162_Map.tif', 'ESA_WorldCover_10m_2020_v100_S12E156_Map.tif', 'ESA_WorldCover_10m_2020_v100_S12E159_Map.tif', 'ESA_WorldCover_10m_2020_v100_S12E162_Map.tif', 'ESA_WorldCover_10m_2020_v100_S12E165_Map.tif']
8


In [18]:
#select all files in folder, and make new raster
#saves new tiled layer to the downloaded data folder (Forest Impact Files)
#this code should still work regardless of year, the file names just need to have ESA in them

print("Processing ESA Land Cover Mosaic")
print("Number of tiles to be Mosaicked: " + str(len(ESA_List)))
arcpy.management.MosaicToNewRaster(ESA_List, downloaded_data, "esa2020_si", None, "8_BIT_UNSIGNED", None, 1, "LAST", "FIRST")   
   
print("Land Cover Mosaic Completed!")


Processing ESA Land Cover Mosaic
Number of tiles to be Mosaicked: 8
Land Cover Mosaic Completed!


<a id= "movelayers"></a>


## Moving downloaded data to gdb and saving as new file names

within the loop below we will be :  
    1. selecting layers in downloaded data folder  
    2. checking and then converting to projected coordinate system in they need  
    3. saving new projected layer to geodatabase folder. 


In [11]:
#list all files in downloaded data folder to check everything downloaded ok

#set workspace to downloaded data file
arcpy.env.workspace = downloaded_data

#list feature classes
featureclasses_downloaded = arcpy.ListFeatureClasses()
print(featureclasses_downloaded)

#list rasters
downloaded_rasters = arcpy.ListRasters()
print(downloaded_rasters)

['osm_SI_roads_p.shp', 'Solomons_country_p.shp', 'solomons_line.shp']
['esa2020_si', 'ESA_WorldCover_10m_2020_v100_S09E153_Map.tif', 'ESA_WorldCover_10m_2020_v100_S09E156_Map.tif', 'ESA_WorldCover_10m_2020_v100_S09E159_Map.tif', 'ESA_WorldCover_10m_2020_v100_S09E162_Map.tif', 'ESA_WorldCover_10m_2020_v100_S12E156_Map.tif', 'ESA_WorldCover_10m_2020_v100_S12E159_Map.tif', 'ESA_WorldCover_10m_2020_v100_S12E162_Map.tif', 'ESA_WorldCover_10m_2020_v100_S12E165_Map.tif', 'si_dem']


In [17]:
for layer in featureclasses_downloaded:
    # Get the full path of the input layer
    input_layer_path = os.path.join(downloaded_data, layer)
    
    # Generate a new name for the output layer
    output_layer_name = arcpy.ValidateTableName(layer, gdb_workspace)
    
    # Set the output layer path in the geodatabase
    output_layer_path = os.path.join(gdb_workspace, output_layer_name)
    
    # Project the layer to the target spatial reference
    arcpy.management.Project(input_layer_path, output_layer_path, project_coord)
    
    print(f"Layer '{layer}' projected and saved to '{output_layer_path}'")

print("Conversion complete.")

Layer 'osm_SI_roads_p.shp' projected and saved to 'C:/Users/jc446202/OneDrive - James Cook University (1)/ACIAR_spatial/modified_dat/Forest_ExPot.gdb\osm_SI_roads_p_shp'
Layer 'Solomons_country_p.shp' projected and saved to 'C:/Users/jc446202/OneDrive - James Cook University (1)/ACIAR_spatial/modified_dat/Forest_ExPot.gdb\Solomons_country_p_shp'
Layer 'solomons_line.shp' projected and saved to 'C:/Users/jc446202/OneDrive - James Cook University (1)/ACIAR_spatial/modified_dat/Forest_ExPot.gdb\solomons_line_shp'
Conversion complete.


In [13]:
# want to move the raster layers to the geodatabase
#but only want to move the combined esa layers (esa2020_si) and the dem layer 
rasters_to_move = "esa2020_si", "si_dem"
print(rasters_to_move)

('esa2020_si', 'si_dem')


In [14]:
#project then copy to geodatabase
#this gets the rasters we want to move, projects them, and then copies them to geodatabase (ExPot)
#
for layer in rasters_to_move:
    # Get the full path of the input layer
    input_layer_path = os.path.join(downloaded_data, layer)
    
    # Generate a new name for the output layer
    output_layer_name = arcpy.ValidateTableName(layer, gdb_workspace)
    
    # Set the output layer path in the geodatabase
    output_layer_path = os.path.join(gdb_workspace, output_layer_name)
    
    # Project the layer to the target spatial reference
    arcpy.management.ProjectRaster(input_layer_path, output_layer_path, project_coord)
    
    print(f"Layer '{layer}' projected and saved to '{output_layer_path}'")

print("Conversion complete.")

Layer 'esa2020_si' projected and saved to 'C:/Users/jc446202/OneDrive - James Cook University (1)/ACIAR_spatial/modified_dat/Forest_ExPot.gdb\esa2020_si'
Layer 'si_dem' projected and saved to 'C:/Users/jc446202/OneDrive - James Cook University (1)/ACIAR_spatial/modified_dat/Forest_ExPot.gdb\si_dem'
Conversion complete.


In [16]:
#set workspace to geodatabase now:
arcpy.env.workspace = gdb_workspace
gdb_shapefiles = arcpy.ListFeatureClasses()
print(gdb_shapefiles)

['osm_SI_roads_p_shp', 'Solomons_country_p_shp', 'solomons_line_shp']


In [19]:
#Renaming the files in the gdb folder


#changing downloaded shapefile to a consistent name
country_polygon = 'Solomons_country_p_shp'            #this is where you will change the name of the shapefile you have downloaded (The right hand side of the "=" sign) 
country_line = 'solomons_line_shp'
roads = 'osm_SI_roads_p_shp'

#change DEM layer to a consistent name
DEM = 'si_dem'




<a id= "ForestCover"></a>

# 3. Forest Cover

Want to reclassify the raster to forest or non forest using the ESA World Cover ("esa2020_si")  

Forest = "Forest" (10) and "Mangrove" (95)  

Non-Forest = all other layers  

**resulting layer will be:**  

1 = forest and mangrove habitat  
0 = all other layers  

Original ESA layers are:  

10 = Tree Cover   
20 = Shrubland   
30 = Grassland    
40 = Cropland     
50 = Built Up     
60 = Bare/Sparse Vegetation    
70 = Snow and Ice    
80 = Permanent Water Bodies    
90 = Herbaceous Wetland      
95 = Mangroves    


In [20]:
#reclassify land cover

#might need to run the checkoutExtension
arcpy.CheckOutExtension("Spatial")

# Define the input raster dataset
input_raster = "esa2020_si"

# Define the reclassification table
reclassification_table = "10 1;20 0;30 0;40 0;50 0;60 0;80 0;90 0;95 1"

# Define the output raster dataset path
output_raster = os.path.join(gdb_workspace, "forest_class")

# Run the Reclassify tool
arcpy.ddd.Reclassify(input_raster, "Value", reclassification_table, output_raster)

#check the license back in when done
arcpy.CheckInExtension("Spatial")

'CheckedIn'

<a id= "AccessLayer"></a>

# 4. Access Layer 


Access will be dictacted by four aspects:  
A. **Distance to coast**   
    villages and locals are not the primary culprit of deforestation. 
    often logging companies build temporary logging stations along the coast to access forest areas
    Currently the distance measure is set to a linear metric
    Can probably change to another relationship, for example 
        you might want a gaussian curve, where access (use) is low very very near the coast, 
        increases when a little bit inland, 
        and then decreases rapidly the further inland you go.  
        but tbd if we want to do this 
    
B. **Road access**     
    roads are a primary driver of deforestation  
    1 km road buffers are set as default.  
    
C. **Slope**  
steep slopes less likely to be deforested compared to flat land   
        > 31 degree slope = not accessible  
        21 - 31 = moderately accessbiel  
        < 21 = very accessible  
        
D. **Elevation**  
Areas >400 m are not allowed to be forested based on SI forestry policy. 
        > 400 m = not accessbile (0)
        < 400 m = accessbile (100)
        while I know that it not always followed, when combined, the distance to coast, and slope layers do allow use of forest resources above 400m 

E. **Adjacent to previously modified land**     
    areas adjacent to modified land are more likely to be deforested. 
    i. deforestation, agriculture, urban areas all get combined into a measure of 'modification'  
    ii. modified areas get aggregated into larger polygons, so there is not so many small polys to work with  
    iii. variable buffer areas related to modified land size get created. where smaller areas have smaller buffers, and larger areas have larger buffers.
    

<a id= "distcoast"></a>

## Distance to Coast

In [23]:
#might need to run the checkoutExtension - if running in VSC
arcpy.CheckOutExtension("Spatial")

'CheckedOut'

In [26]:
#4.A Distance to Coast



#euclidian distance measures 
out_distance_raster = arcpy.sa.EucDistance(country_line, 25000, 50, None, "PLANAR", None, None); out_distance_raster.save("d_coast")
#NOTE - it is possible to change this relationship from linear to non linear
#would need to use rescale by function tool

#extract for just land areas
out_raster = arcpy.sa.ExtractByMask("d_coast", country_polygon); out_raster.save("d_coast_land")

#scale distance to coast_land
out_raster = arcpy.sa.RescaleByFunction("d_coast_land", "LINEAR # # # # # #", 100, 0); out_raster.save("d_coast_scale")


<a id= "roads"></a>

## Road Buffers

In [27]:
print(roads)

osm_SI_roads_p_shp


In [28]:
#4.B Road Buffers

#create 1km buffers around roads. select all road types.
#written as 1k bc thats the TOTAL buffer distance (500 m on each side = 1km total)
arcpy.analysis.Buffer(roads, "SI_roads_buff1k", road_buff_dist, "FULL", "ROUND", "ALL", None, "PLANAR") # this is dissolved all features 

#if you havent specified road_buff_dist in the code above
#arcpy.analysis.Buffer("osm_SI_roads_p", "SI_roads_buff1k", "1 Kilometers", "FULL", "ROUND", "ALL", None, "PLANAR")


<Result 'C:/Users/jc446202/OneDrive - James Cook University (1)/ACIAR_spatial/modified_dat/Forest_ExPot.gdb\\SI_roads_buff1k'>

In [29]:
print(country_polygon)

Solomons_country_p_shp


In [31]:
#clip buffers to land, bc some coastal roads have buffer that extend into the sea
arcpy.analysis.Clip("SI_roads_buff1k", country_polygon, "roads_buff1k", None)

#delete first buffer file because its not useful 
arcpy.management.Delete("SI_roads_buff1k")
#rename clipped file something sensible

<Result 'true'>

In [33]:
#create new field in attribute table called VALUE (to assign Hazard value to buffer area)
arcpy.management.AddField('roads_buff1k',"Value", 'SHORT',3,"","","Value","","","")

#Set road buffer value to 100
#change value to 100 . this is the scaled data. If you want a different scale change value from 100 to whatever you want
fc = "roads_buff1k"
field = ['Value']

with arcpy.da.UpdateCursor(fc, field) as cursor:
	for row in cursor:
        	if row[0] == None: 
            		row[0] = 100
            		cursor.updateRow(row) #NOTE INDENTS ARE VERY IMPORTANT
      

In [35]:
#convert to buffer layer to raster. 
#want to be the same spatial extent at the DEM raster - which should be already loaded (31m)
arcpy.conversion.PolygonToRaster('roads_buff1k', 'Value','roads_buff1k_rast', 'CELL_CENTER', 'NONE','si_dem', 'BUILD')


<Result 'C:/Users/jc446202/OneDrive - James Cook University (1)/ACIAR_spatial/modified_dat/Forest_ExPot.gdb\\roads_buff1k_rast'>

<a id= "slope"></a>

## Slope

In [36]:
#4.C Slope

#Calculate slope from DEM
arcpy.ddd.Slope(DEM, "Slope", "DEGREE", 1, "PLANAR", "METER")

#Classify slope, and use scaled values (0-100)
#0-20 degree = 100 (highest access)
#21-31 degree = 50 (medium access)
#>31 degree = 0 (no access)
arcpy.ddd.Reclassify("Slope", "VALUE", "0 20 100;20 31 50;31 90 0", "slope_class", "DATA")

<Result 'C:/Users/jc446202/OneDrive - James Cook University (1)/ACIAR_spatial/modified_dat/Forest_ExPot.gdb\\slope_class'>

<a id= "elevation"></a>

## Elevation

1. identify min and max values of elevation (so that you can apply this code to any location)
2. classify areas based on < 400 m > elevation

In [37]:
#4.D Elevation

# identify min and max values of elevation

#Set parameters
elev_min_output = arcpy.management.GetRasterProperties(DEM, 'MINIMUM',"") #extracts minimum value from raster
elev_min = int(elev_min_output.getOutput(0)) #assigns that value to an object, and converts to a integer


elev_max_ouput = arcpy.management.GetRasterProperties(DEM, 'MAXIMUM',"")
elev_max = int(elev_max_ouput.getOutput(0))

# Assign elevation value and access values
elevation_cutoff     # the elevation where no forestry is allowed above that value. Default = 400 m. this is assigned earlier in "Set Parameters" section
access_val = 100
no_access_val = 0

print(elev_min)
print(elev_max)
print(elevation_cutoff)
print(access_val)
print(no_access_val)


-9
2326
400
100
0


In [38]:
#create string to input into reclassify 
#this will allow you to use this code for any elevation layer and not be specific to 

elevation_class_val = '{}, {}, {}; {}, {}, {}'.format(elev_min, elevation_cutoff, access_val, elevation_cutoff, elev_max, no_access_val)
elevation_class_val

'-9, 400, 100; 400, 2326, 0'

In [39]:

#Classify elevation, and use scaled values (0-100)
# < 400 m = 100 (highest access)
# > 400 m = 0 (no access)
arcpy.ddd.Reclassify(DEM, "VALUE", elevation_class_val, "elevation_class", "DATA")


<Result 'C:/Users/jc446202/OneDrive - James Cook University (1)/ACIAR_spatial/modified_dat/Forest_ExPot.gdb\\elevation_class'>

<a id= "modifiedland"></a>

## Adjacent to Modified Land

extracting areas of modified land and creating buffers around them
in theory, forests that are within 200m of modified land (urban areas, agriculture) have a greater chance of being used and/or modified.

In [40]:
#i. convert Land Cover Layer to Polygon
arcpy.conversion.RasterToPolygon('esa2020_si','LandCover_poly', 'NO_SIMPLIFY', 'Value', 'SINGLE_OUTER_PART','',)


<Result 'C:/Users/jc446202/OneDrive - James Cook University (1)/ACIAR_spatial/modified_dat/Forest_ExPot.gdb\\LandCover_poly'>

In [41]:
#ii. add Land Cover Type to new field "LandCover"

#add land cover class to attribute tables.
arcpy.management.AddField('LandCover_poly','LandCover', 'TEXT')

#input values for new fiedl

fc = "LandCover_poly"
fields = ['gridcode', 'LandCover']

with arcpy.da.UpdateCursor(fc, fields) as cursor:
    #for each row, evaluate the 'gridcode' field value (index position of 0)
    #and update 'LandCover' (index position of 1)
    for row in cursor:
        if (row[0] == 10):
            row[1] = "Forest"
        elif (row[0] == 20):
            row[1] = "Shrubland"            
        elif (row[0] == 30):
            row[1] = "Grassland"            
        elif (row[0] == 40):
            row[1] = "Cropland"            
        elif (row[0] == 50):
            row[1] = "Built Up"            
        elif (row[0] == 60):
            row[1] = "Bare/Sparse Vegetation"            
        elif (row[0] == 70):
            row[1] = "Snow and Ice"            
        elif (row[0] == 80):
            row[1] = "Permanent Water Bodies"            
        elif (row[0] == 90):
            row[1] = "Herbaceous Wetland"
        elif (row[0] == 95):
            row[1] = "Mangroves"      
        
        #Update cursor with the updated list
        cursor.updateRow(row)
            
            
            

In [42]:
#vi select deforested, grassland, cropland, built up, bare/sparse vegetation area polygons to create a "modified land" layer
#dissolve them to make the layer more manageable
#delete the initial modified land and just keep the dissolved layers


arcpy.management.SelectLayerByAttribute("LandCover_poly", "NEW_SELECTION", "LandCover = 'Shrubland' Or \
                                                                                 LandCover = 'Grassland' Or \
                                                                                 LandCover = 'Cropland' Or \
                                                                                 LandCover = 'Built Up' Or \
                                                                                 LandCover = 'Bare/Sparse Vegetation'", \
                                                                                 None)

arcpy.management.CopyFeatures('LandCover_poly', 'modified_land')

#clear selections
arcpy.management.SelectLayerByAttribute("LandCover_poly", "CLEAR_SELECTION")

#dissolve all modified land polygons by land type
arcpy.management.Dissolve("modified_land", "modified_land_types", "LandCover", multi_part= "MULTI_PART")

#delete intermediate files 
arcpy.management.Delete("modified_land")

<Result 'LandCover_poly_Layer2'>

In [21]:
#xi. create buffer of modified land areas 

#commented out because it takes forever to run. be warned.
#arcpy.analysis.Buffer("modified_land_types", "modified_land_Buff200m", "200 Meters", "OUTSIDE_ONLY", "ROUND", "ALL", None, "PLANAR")
#200 meter buffer distances. 



<Result 'C:/Users/jc446202/OneDrive - James Cook University (1)/ACIAR_spatial/modified_dat/Forest_Impact.gdb\\modified_land_agg_Buff200m'>

In [16]:
#create new field in attribute table called VALUE (to assign Hazard value of 100 to buffer area)

arcpy.management.AddField('modified_land_buffer200m',"Value", 'SHORT',3,"","","Value","","","")

#Set modified land buffer value to 100
fc = "modified_land_buffer200m"
field = ['Value']

with arcpy.da.UpdateCursor(fc, field) as cursor:
	for row in cursor:
            #none = <NULL> which is what the empty field will be
        	if row[0] == None:  
            		row[0] = 100
            		cursor.updateRow(row) #NOTE INDENTS ARE VERY IMPORTANT

In [20]:
#need to clip this bc some of the buffers go outside of the land area into the sea. 

arcpy.analysis.Clip("modified_land_buffer200m", country_polygon, "modified_land_buff200m_clip", None)

#delete intermediate file
arcpy.management.Delete("modified_land_buffer200m")

#keep clipped file and rename it
arcpy.management.Rename("modified_land_buff200m_clip", "areas_adjacent_to_modified_land")

<Result 'C:/Users/jc446202/OneDrive - James Cook University (1)/ACIAR_spatial/modified_dat/Forest_ExPot.gdb\\areas_adjacent_to_modified_land'>

In [21]:
#xii. convert buffers to raster. 

arcpy.conversion.PolygonToRaster("areas_adjacent_to_modified_land", "Value", "areas_adjacent_to_modified_land_rast", "CELL_CENTER", "NONE", "esa2020_si", "BUILD")


<Result 'C:/Users/jc446202/OneDrive - James Cook University (1)/ACIAR_spatial/modified_dat/Forest_ExPot.gdb\\areas_adjacent_to_modified_land_rast'>

<a id= "combineaccess"></a>

## Combine Access Layers


In [26]:
#Combine Access layers together to get total scaled access:
#remember each needs to be scaled to similar scales before combining bc theyre in different units. thats already been completed in the code

#need to use **cell statistics** so the entire area gets calculated and not just where all three rasters intersect
# must select "UNION of INPUTS" for environment. 

with arcpy.EnvManager(extent="MAXOF"): #this means UNION of inputs
    out_raster = arcpy.ia.CellStatistics("d_coast_scale;roads_buff1k_rast;slope_class;elevation_class;areas_adjacent_to_modified_land_rast", "SUM", "DATA", "SINGLE_BAND", 90, "AUTO_DETECT"); out_raster.save("forest_access_all")
    

In [27]:
#rescale by function
#need to use this tool rather than the normal (x-min)/(min-max) because that equation needs values to be integers 
#this layer uses continuous layers (distance from coast) so you can have non integer values

out_raster = arcpy.sa.RescaleByFunction("forest_access_all", "LINEAR # # # # # #", 0, 100); out_raster.save("forest_access_scale")



cannot use raster calculator in python coding unless using the GUI
when using map algebra, it thinks things are either a string or number (int, float etc)
so for a raster, where you are calling an object, you need to nominate it as a Raster first
https://pro.arcgis.com/en/pro-app/2.8/help/analysis/spatial-analyst/mapalgebra/a-quick-tour-of-using-map-algebra.htm


<a id= "CombineFC_Access"></a>


# 5. Combing Land Cover + Access

Need to combine access * land cover  
access is the SUM of layers: distance, road, slope, adjacent to modification

this needs to be MULTIPLIED by forest_scale layer.  
Because access hazard means nothing if its in non forest areas (areas where never was forest, and areas of deforestation) 
non forest raster value = 0  
forest value = 100  
this will mean that access is ONLY measurable in forested areas which is our resournce for this hazard layer. 



In [28]:
#forest access * land use 

out_raster = Raster("forest_class") * Raster("forest_access_scale"); out_raster.save("forest_potential_exploitation")
print("completed forest PotExp layer")


completed forest PotExp layer


In [30]:
#identify min and max values of forest_impact

forest_potexp_min_output = arcpy.management.GetRasterProperties('forest_potential_exploitation', 'MINIMUM',"") #extracts minimum value from raster
forest_potexp_min = int(forest_potexp_min_output.getOutput(0)) #assigns that value to an object, and converts to a integer
print(forest_potexp_min)

forest_potexp_max_ouput = arcpy.management.GetRasterProperties('forest_potential_exploitation', 'MAXIMUM',"")
forest_potexp_max = float(forest_potexp_max_ouput.getOutput(0))
print(forest_potexp_max)


0
100.000015258789


In [31]:
#checking object type to confirm integer
print(type(forest_potexp_min))
print(type(forest_potexp_max))


<class 'int'>
<class 'float'>


In [34]:
#SCALE the forst impact layer 

outraster = ((Raster("forest_potential_exploitation") - forest_potexp_min)/(forest_potexp_max - forest_potexp_min))*100; outraster.save("forest_potexp_scale")

print("completed forest potential exploitation scaled layer")

completed forest impact scale layer


In [32]:
arcpy.CheckInExtension("Spatial")

'CheckedIn'

In [34]:
features = arcpy.ListFeatureClasses()
print(features)

['osm_SI_roads_p_shp', 'Solomons_country_p_shp', 'solomons_line_shp', 'roads_buff1k', 'LandCover_poly', 'modified_land', 'modified_land_types', 'areas_adjacent_to_modified_land']


In [37]:
#convert non projected data to projected data (PDC mercator)

#Loop through the list
for infc in features:                                            
    #determine if input has a defined coordinate system
    spatial_ref = arcpy.Describe(infc).spatialReference

    # If the spatial reference is not the correct projected coordinate system
    if spatial_ref.name == project_coord.name:
        print("{} is already in correct projection".format(infc))
    
    else:
        # determine new output feature class (outfc) path and name. 
        #this saves to same gdb, saves as same name, and then add "_p" to the end to indicate its projected
        outfc = os.path.join(gdb_workspace, infc+'_p')
        
        # set projected coord system to use
        outCS = project_coord
        
        #run project tool
        arcpy.management.Project(infc, outfc, outCS)
        print("{} has been  converted and is now: {}".format(infc, outfc))

osm_SI_roads_p_shp is already in correct projection
Solomons_country_p_shp is already in correct projection
solomons_line_shp is already in correct projection
roads_buff1k is already in correct projection
LandCover_poly is already in correct projection
modified_land has been  converted and is now: C:/Users/jc446202/OneDrive - James Cook University (1)/ACIAR_spatial/modified_dat/Forest_ExPot.gdb\modified_land_p
modified_land_types has been  converted and is now: C:/Users/jc446202/OneDrive - James Cook University (1)/ACIAR_spatial/modified_dat/Forest_ExPot.gdb\modified_land_types_p
areas_adjacent_to_modified_land has been  converted and is now: C:/Users/jc446202/OneDrive - James Cook University (1)/ACIAR_spatial/modified_dat/Forest_ExPot.gdb\areas_adjacent_to_modified_land_p
